# NARM+




In [114]:
#Importing functions.
import pandas
import csv
from collections import defaultdict
from collections import namedtuple
import itertools
import math
import numpy as np
import pandas
import random
from sklearn.metrics import explained_variance_score
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


In [115]:
# Create sessions
sample = pandas.read_pickle("./data/smallTaoBao.pkl")
sample['SESSION'] = pandas.to_datetime(unpickled_smallTaoBao['TIMESTAMP'],unit='s').dt.date
sample.head()

,USERID,PRODUCTID,CATEGORYID,ACTION,TIMESTAMP,SESSION
0,1,2268318,2520377,pv,1511544070,2017-11-24
1,1,2333346,2520771,pv,1511561733,2017-11-24
2,1,2576651,149192,pv,1511572885,2017-11-25
3,1,3830808,4181361,pv,1511593493,2017-11-25
4,1,4365585,2520377,pv,1511596146,2017-11-25


In [116]:
print("Average number of sessions per user")
unpickled_smallTaoBao.groupby('USERID')['SESSION'].nunique().mean()

Average number of sessions per user


7.2

In [117]:
print("Average number of clicks per session")
sample.groupby(['USERID', 'SESSION'])['ACTION'].count().mean()

Average number of clicks per session


14.126388888888888

In [118]:
user1 = sample.loc[unpickled_smallTaoBao['USERID'] == 1]
user1.head()

,USERID,PRODUCTID,CATEGORYID,ACTION,TIMESTAMP,SESSION
0,1,2268318,2520377,pv,1511544070,2017-11-24
1,1,2333346,2520771,pv,1511561733,2017-11-24
2,1,2576651,149192,pv,1511572885,2017-11-25
3,1,3830808,4181361,pv,1511593493,2017-11-25
4,1,4365585,2520377,pv,1511596146,2017-11-25


In [121]:
# Create nested list of sessions and items per user
userBase = sample.groupby(['USERID', 'SESSION'])['PRODUCTID'].apply(list).groupby('USERID').apply(list)
print(userBase[1])

[[2268318, 2333346], [2576651, 3830808, 4365585, 4606018, 230380], [3827899, 3745169, 1531036], [2266567, 2951368, 3108797, 1338525, 2286574], [5002615, 2734026, 5002615, 3239041, 4615417, 4152983, 266784, 46259, 266784, 4092065, 1305059], [2791761, 3239041, 46259, 4973305, 2087357, 3157558], [2087357, 4170517, 1340922, 3911125, 4170517, 3682069, 4954999, 79715, 4666650], [1323189, 4198227, 4954999], [2041056, 3219016, 2104483, 2028434, 3219016, 2278603, 929177], [4954999, 818610, 271696, 568695]]


In [120]:
# More efficient create examples function
# A simple way to define a class is using namedtuple.
Example = namedtuple("Example", ["userID", "history", "inputs", "target"])

def f(userid, sessions, train):
    if train:
        object_train = Example(userID = userid, history = sessions[:-2], inputs = sessions[-2], target = sessions[-2][1:])
        return object_train
    else:
        return Example(userID = userid, history = sessions[:-1], inputs = sessions[-1], target = sessions[-1][1:])

def createExamples(userBase):
    ''' Create training and testing set '''
    userBase = pandas.DataFrame(userBase)
    userBase.reset_index(level = 0, inplace = True)
    trainData = userBase.apply(lambda x: f(x['USERID'], x['PRODUCTID'], True), axis = 1).tolist()
    testData = userBase.apply(lambda x: f(x['USERID'], x['PRODUCTID'], False), axis = 1).tolist()
    return trainData, testData

trainData, testData = createExamples(userBase)
print(trainData[0])
print('')
print(testData[0])
    
    

Example(userID=1, history=[[2268318, 2333346], [2576651, 3830808, 4365585, 4606018, 230380], [3827899, 3745169, 1531036], [2266567, 2951368, 3108797, 1338525, 2286574], [5002615, 2734026, 5002615, 3239041, 4615417, 4152983, 266784, 46259, 266784, 4092065, 1305059], [2791761, 3239041, 46259, 4973305, 2087357, 3157558], [2087357, 4170517, 1340922, 3911125, 4170517, 3682069, 4954999, 79715, 4666650], [1323189, 4198227, 4954999]], inputs=[2041056, 3219016, 2104483, 2028434, 3219016, 2278603, 929177], target=[3219016, 2104483, 2028434, 3219016, 2278603, 929177])

Example(userID=1, history=[[2268318, 2333346], [2576651, 3830808, 4365585, 4606018, 230380], [3827899, 3745169, 1531036], [2266567, 2951368, 3108797, 1338525, 2286574], [5002615, 2734026, 5002615, 3239041, 4615417, 4152983, 266784, 46259, 266784, 4092065, 1305059], [2791761, 3239041, 46259, 4973305, 2087357, 3157558], [2087357, 4170517, 1340922, 3911125, 4170517, 3682069, 4954999, 79715, 4666650], [1323189, 4198227, 4954999], [2041

In [51]:
# This will probably take very long for entire dataset, code above does the same
# IDs = list(sample.USERID.unique())
# sessions = sorted(list(unpickled_smallTaoBao.SESSION.unique()))
# userBase = defaultdict(list)

# for ID in IDs:
#     userData = sample.loc[unpickled_smallTaoBao['USERID'] == ID]
    
#     #Verschil tussen historical data and current session.
    
#     for session in sessions:
#         userSession = userData.loc[userData['SESSION'] == session]
#         userData_PRODUCTLIST = userSession['PRODUCTID'].values.tolist()
        
#         if len(userData_PRODUCTLIST) > 0:
#             userBase[ID].append(userData_PRODUCTLIST)
    
    

# print(userBase[1])

In [7]:
# # DON'T RUN THIS YET, NEEDS REWORKING

# # Create dataset of examples

# # A simple way to define a class is using namedtuple.
# Example = namedtuple("Example", ["userID", "history", "inputs", "target"])

# def createExamples(userBase, IDs):
#     # convert raw data into examples and create dataset
    
#     #TODO: create train and testdata
#     #Example = userID, history, inputs, targets
    
#     trainData = []
#     testData = []
    
    
    
#     for ID in IDs:
#         userData = userBase[ID]
#         historyTrain = userData[:-2]
#         inputsTrain = userData[len(userData) -2]
#         trainData.append(Example(userID = ID, history = historyTrain, inputs=inputsTrain[:-1],target=inputsTrain[1:]))
        
#         historyTest = userData[:-1]
#         inputsTest = userData[len(userData) - 1]
#         testData.append(Example(userID = ID, history = historyTest, inputs=inputsTest[:-1], target=inputsTest[1:]))
        
#     return trainData, testData

# trainData, testData = createExamples(userBase, IDs)
# print(trainData[0])
# print(testData[0])


Example(userID=1, history=[[2268318, 2333346], [2576651, 3830808, 4365585, 4606018, 230380], [3827899, 3745169, 1531036], [2266567, 2951368, 3108797, 1338525, 2286574], [5002615, 2734026, 5002615, 3239041, 4615417, 4152983, 266784, 46259, 266784, 4092065, 1305059], [2791761, 3239041, 46259, 4973305, 2087357, 3157558], [2087357, 4170517, 1340922, 3911125, 4170517, 3682069, 4954999, 79715, 4666650], [1323189, 4198227, 4954999]], inputs=[2041056, 3219016, 2104483, 2028434, 3219016, 2278603], target=[3219016, 2104483, 2028434, 3219016, 2278603, 929177])
Example(userID=1, history=[[2268318, 2333346], [2576651, 3830808, 4365585, 4606018, 230380], [3827899, 3745169, 1531036], [2266567, 2951368, 3108797, 1338525, 2286574], [5002615, 2734026, 5002615, 3239041, 4615417, 4152983, 266784, 46259, 266784, 4092065, 1305059], [2791761, 3239041, 46259, 4973305, 2087357, 3157558], [2087357, 4170517, 1340922, 3911125, 4170517, 3682069, 4954999, 79715, 4666650], [1323189, 4198227, 4954999], [2041056, 3219

In [8]:
# HELPER FUNCTIONS

# function to yield one example at a time
def get_examples(data, shuffle=True, **kwargs):
    """Shuffle data set and return 1 example at a time (until nothing left)"""
    if shuffle:
#         print("Shuffling training data")
        random.shuffle(data)  # shuffle training data each epoch
    for example in data:
        yield example
    
# function to prepare an example for usage by the model
def prepare_example(example):
    """
    Turn an example into tensors of inputs and target.
    """
    v = torch.FloatTensor([example.userID])
    v = v.to(device)
    
    w = torch.FloatTensor([example.history])
    w = w.to(device)
    
    x = torch.FloatTensor([example.inputs])
    x = x.to(device)

    y = torch.FloatTensor([example.target])
    y = y.to(device)

    return v, w, x, y

# function to yield a (mini-)batch
def get_minibatch(data, batch_size=25, shuffle=True):
    """Return minibatches, optional shuffling""" 
    if shuffle:
#         print("Shuffling training data")
        random.shuffle(data)  # shuffle training data each epoch

    batch = []

    # yield minibatches
    for example in data:
        batch.append(example)

        if len(batch) == batch_size:
            yield batch
            batch = []    
        # in case there is something left
        if len(batch) > 0:
            yield batch

# function to make the (mini-)batch ready for usage by the model
def prepare_minibatch(mb):
    """
    Minibatch is a list of examples.
    This function converts returns
    torch tensors to be used as input/targets.
    """
    batch_size = len(mb)
    x = [ex.inputs for ex in mb]
    x = torch.FloatTensor(x)
    x = x.to(device)

    y = [ex.target for ex in mb]
    y = torch.FloatTensor(y)
    y = y.to(device)

    return x, y

# simple evaluation function
def simple_evaluate(model, data, prep_fn=prepare_example, **kwargs):
    """Explained Variance Score of a model on given data set."""
    model.eval()  # disable dropout (explained later)
    targets = []
    predictions = []

    for example in data:

        # convert the example input and label to PyTorch tensors
        targets.append(example.target)
        x, target = prepare_example(example)

        # forward pass
        # get the output from the neural network for input x
        with torch.no_grad():
            output = model(x)
        # output shape: (batch, output_size)
        prediction = output[0].tolist()
        predictions.append(prediction)
            
    score = explained_variance_score(targets, predictions, multioutput='variance_weighted')

    return score, None

In [124]:
# Custom NN

class NarmPlus(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_size, output_dim, num_layers, activation_fn=nn.RReLU(),num_items):
        super(NarmPlus, self).__init__()
        self.input_dim = input_dim
        self.embedding_dim = embedding_dim
        self.hidden_size = hidden_size
        self.output_dim = output_dim
        self.num_layers = num_layers
        self.hidden_state_dim = (num_layers, 1, hidden_size)
        self.hidden_state_size = num_layers * hidden_size
        self.UserEmbedding = nn.Embedding(num_users, embedding_dim)
        self.ItemEmbedding = nn.Embedding(num_items, embedding_dim)
        self.ActivationFn = activation_fn
        self.UserHistory = nn.Linear(embedding_dim, self.hidden_state_size)
        self.Local = nn.GRU(,batch_first=True)
        self.Global = nn.GRU(,batch_first=True)
        self.Softmax = nn.Softmax()
        self.Decoder = nn.Bilinear(embedding_dim, , output_dim)
        # TODO: find input and output size of A1 and A2
        self.A1 = nn.Linear()
        self.A2 = nn.Linear()
        self.loss = top1
        
    def forward(self,x):
        user, history, inputs = x
        # user shape (1)
        # history shape (history_length)
        # current shape (seq_length,1)
        item_embeds_h = self.ItemEmbedding(history)
        # item_embeds_h shape (history_length, embedding_size)
        user_embed = self.UserEmbedding(user)
        # user_embed shape (1, embedding_size)
        dense = self.ActivationFn(self.UserHistory(item_embeds_h))
        # dense shape (history_length, hidden_state_size)
        alpha1 = self.Softmax(torch.matmul(user_embed,torch.transpose(dense, 0, 1)))
        # alpha shape (history_length)
        profile = torch.sum(torch.mul(alpha1,torch.transpose(dense,0,1)),1)
        # profile shape (hidden_state_size)
        # reshape to correct hidden state dimensions
        h_0 = torch.reshape(profile,*self.hidden_state_dim)
        
        # add a batch dimension to the front, necessary for GRU
        inputs = inputs[None,:,:]
        
        out_local, _ = self.Local(inputs,h_0)
        out_global, _ = self.Global(inputs,h_0)
        # out shape (batch_size, seq_length, hidden_size), containing hidden state output for every step
        
        A1 = self.A1()
        alpha2 = self.ActivationFn(self.A1())
        
        c_local = torch.cumsum(torch.mul(alpha2,torch.transpose(out_local[1,:,:],0,1)),1)
        # c_local shape (hidden_size, sequence length)
        
        
        c = torch.cat(c_global, c_local, dim=2)
        out = self.Decoder()
        output = self.Softmax(out)
        return output
    
    def alpha2(self,H):
        # H shape (batch, sequence, hidden size)
        H = H.squeeze()
        # H shape (sequence, hidden size)
        s = H.shape
        alpha = torch.ones(s, dtype=torch.float64, device=device)
        
        for t in range(H.shape[1]):
            A1 = self.A1(H[t])
            for j in range(t):
                
            alpha[1,t] = torch.add(A1,A2)
    
    def top1(self, yhat):\n",
        ''' Top1 loss, yhat is vector with softmax probabilities '''
        # Not sure if you can just call backward to this, but I think it should work. Code from:
        # https://github.com/mquad/hgru4rec/blob/master/src/hgru4rec.py
        yhatT = torch.transpose(yhat, 0, 1)
        loss = torch.mean(torch.mean(nn.sigmoid( - torch.diag(yhat) + yhatT) + nn.sigmoid(yhat ** 2), dim = 0) - nn.sigmoid(T.diag(yhat ** 2)))
        return loss
        

SyntaxError: non-default argument follows default argument (<ipython-input-124-07df95cea523>, line 4)